In [18]:
from langchain.llms import OpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.utilities import SQLDatabase
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory,ConversationBufferWindowMemory
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain_core.prompts.prompt import PromptTemplate
import langchain 




import os
import ast
import re
import json

from dotenv import load_dotenv
load_dotenv()


True

In [19]:
langchain.debug = True

In [20]:
db = SQLDatabase.from_uri(f"postgresql+psycopg2://{os.environ['user']}:{os.environ['db_password']}@{os.environ['host']}:{os.environ['port']}/{os.environ['db_name']}",sample_rows_in_table_info=1,include_tables=['fund_detail','performance','region','sector'])

## Creating & Saving Tools

In [21]:
def run_query_save_results(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return res


category = run_query_save_results(db, "SELECT category FROM fund_detail")
sector = run_query_save_results(db,"SELECT sector FROM sector")
region = run_query_save_results(db,"SELECT region FROM region")
symbols = run_query_save_results(db,"SELECT symbol FROM fund_detail")

texts = category + sector + region + symbols

with open('../langchain/pronouns.json','w')as f:
    pronouns = {"pronouns":list(set(texts))}
    json.dump(pronouns,f)

few_shots = {
    "List the top 5 equity funds in terms of year to date returns along with their respective percentage return.": "SELECT fund_detail.symbol, performance.fund_return_ytd FROM fund_detail INNER JOIN performance ON performance.symbol = fund_detail.symbol WHERE fund_detail.asset_class LIKE '%Equity%' OR fund_detail.asset_class LIKE '%Stock%' AND performance.fund_return_ytd IS NOT NULL ORDER BY performance.fund_return_ytd DESC LIMIT 5;",
    "List the top 5 fixed income funds in terms of year to date returns along with their percentage return.": "SELECT fund_detail.symbol, performance.fund_return_ytd FROM fund_detail INNER JOIN performance ON performance.symbol = fund_detail.symbol WHERE (fund_detail.asset_class LIKE '%Money Market%' OR fund_detail.asset_class LIKE '%Bond%' OR fund_detail.asset_class LIKE '%Securities%') AND performance.fund_return_ytd IS NOT NULL ORDER BY performance.fund_return_ytd DESC LIMIT 5;",
    "List the top 5 short term bond funds in terms of average annual return since inception along with their percentage return.": "SELECT fund_detail.symbol, performance.average_annual_fund_return_since_inception FROM fund_detail INNER JOIN performance ON performance.symbol = fund_detail.symbol WHERE fund_detail.asset_class LIKE '%Short-Term Bond%' AND performance.average_annual_fund_return_since_inception IS NOT NULL ORDER BY performance.average_annual_fund_return_since_inception DESC LIMIT 5;",
    "List the top 5 large-cap growth funds in terms of the average annual return for the past 5 years along with their percentage return.": "SELECT fund_detail.symbol, performance.average_annual_fund_return_for_5_year FROM fund_detail INNER JOIN performance ON performance.symbol = fund_detail.symbol WHERE fund_detail.category LIKE '%Large Growth%' AND performance.average_annual_fund_return_for_5_year IS NOT NULL ORDER BY performance.average_annual_fund_return_for_5_year DESC LIMIT 5;",
    "Provide me with a regional breakdown of the fund VWILX asset allocation in percentage point.": "SELECT region.region, region.allocation FROM region WHERE region.symbol = 'VWILX';",
    "Provide me with a sector breakdown of the fund VTMSX asset allocation in percentage point.": "SELECT sector.sector, sector.allocation FROM sector WHERE sector.symbol = 'VTMSX';",
    "List all the funds that has at least 3% of its asset allocated to the Energy sector.": "SELECT sector.symbol FROM sector WHERE sector.allocation >=0.03 AND sector.sector = 'Energy';",
    "List all the funds that has at less than 10% of its asset allocated to the Emerging Market's region.": "SELECT region.symbol FROM region WHERE region.allocation <0.1 AND region.region = 'Emerging Markets';",
    "List all the funds that has less than 5% of its asset allocated to the Pacific region and at least 10% of its asset allocated to the Health Care sector.": "SELECT symbol FROM region WHERE region = 'Pacific' AND allocation < 0.05 INTERSECT SELECT symbol FROM sector WHERE sector = 'Health Care' AND allocation >= 0.1;",
    "Provide me with a product summary of the fund VGWAX.": "SELECT fund_detail.product_summary FROM fund_detail WHERE fund_detail.symbol='VGWAX"
}





with open('../langchain/few_shot_example_query.json','w') as f:
    json.dump(few_shots,f)

In [22]:
API_KEY = os.environ['openai_api_key']

llm = OpenAI(model='gpt-3.5-turbo-instruct',openai_api_key=API_KEY,temperature=0,streaming=True)

embeddings = OpenAIEmbeddings(openai_api_key=API_KEY)

with open('../langchain/pronouns.json','r')as f:
    texts = json.load(f)['pronouns']

vector_db = FAISS.from_texts(texts, embeddings)
retriever = vector_db.as_retriever()

tool_description = """This tool will help you understand the which entity the user is referring to before querying for proper nouns like type of sectors (Health Care), type of regions (North America),symbol of funds (VWEAX) and type of fund categories (Large Growth).
Use it only after you have used tried using the sql_get_similar_examples tool.
"""

fund_type_retriever_tool = create_retriever_tool(
    retriever,
    name="entity_search",
    description=tool_description,
)

with open('../langchain/few_shot_example_query.json','r') as f:
    few_shots = json.load(f)

few_shot_docs = [
    Document(page_content=question, metadata={"sql_query": few_shots[question]})
    for question in few_shots.keys()
]
vector_db = FAISS.from_documents(few_shot_docs, embeddings)
retriever = vector_db.as_retriever()

tool_description = """This tool will help you understand similar examples to adapt them to the user question.
Input to this tool should be the user question.
Always use this tool first.
"""

few_shot_retriever_tool = create_retriever_tool(
    retriever, name="sql_get_similar_examples", description=tool_description
)

custom_tool_list = [few_shot_retriever_tool,fund_type_retriever_tool]


In [23]:
prefix_template = '''You are an agent designed to interact with a PostgreSQL database.
Given an input question, create a syntactically correct PostgresSQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURRENT_DATE function to get the current date, if the question involves "today".

You have access to tools for interacting with the database as well as some extra tools to help you construct accurate queries.
You will ALWAYS FIRST use the tool sql_get_similar_examples to get the similar examples. If the examples are enough to construct the query, you can build it. Otherwise, you will then look at the tables in the database to see what you can query. Then you should use entity_search tool to check that you are querying the right entities before querying the schema of the most relevant tables.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "No such funds." as the answer.

Only use the information returned by the below tools to construct your final answer.

'''

format_instructions_template = """Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}], always look first in sql_get_similar_examples
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: The final answer to the original input question. If the last observation leading up to the final answer does not return any row of result from the database, you should return "No such funds." as your final answer. Otherwise include the symbol of the fund(s) in your final answer and format it in a concise and easy to understand manner with as least words as possible. 
"""

suffix_template = """Begin!

Chat History: {history}
Question: {input}

If the last observation leading up to the final answer does not return any row of result from the database, you should return "No such funds." as your final answer.
{agent_scratchpad}"""  

In [24]:


agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_executor_kwargs={
            "memory": ConversationBufferWindowMemory(
                input_key='input',memory_key="history", return_messages=True,k=1
            ),
            'handle_parsing_errors':"Check your output and make sure it conforms!"
        },
    input_variables = ['input','agent_scratchpad','history'],
    prefix=prefix_template,
    format_instructions=format_instructions_template,
    extra_tools=custom_tool_list,
    suffix=suffix_template,
    top_k=5,
    handle_parsing_errors=True,
)

In [17]:
question_1 = "What are the top 5 equity funds in terms of year to date returns."
question_2 = "What are the corresponding inception date,risk level, expense ratio and average annual return since inception for these funds."
question_3 = "What are the top 5 fixed income funds in terms of year to date returns."
question_4 = "What are the corresponding inception date,risk level, expense ratio and average annual return since inception for these funds."
question_5 = "Find all long-term bond funds and return the top 5 in terms of average annual return since inception."
question_6 = "Find all small blend funds and return the top 5 in terms of average annual return since inception."
question_7 = "Find all the funds that has less than 5% allocated to Emerging Markets region and at least 10% exposure to the health care sector."
question_8 = "Find all the funds that has less than 5% allocated to Europe region and at least 10% of exposure to the health care sector."
question_9 = "Can you provide me with the product summary for the fund VWEAX?"
question_10 = "List all the funds that has at least 10% of its asset allocated to the Pacific region and at least 60% exposure to North America region and at least 5% of its asset allocated to the Consumer Staples sector."

In [9]:
agent_executor.run(question_1)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What are the top 5 equity funds in terms of year to date returns.",
  "history": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "What are the top 5 equity funds in terms of year to date returns.",
  "history": [],
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "You are an agent designed to interact with a PostgreSQL database.\nGiven an input question, create a syntactically correct PostgresSQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostgreSQL. You can order the results to return the most inf

'MGK, VMGAX, VGT, VITAX, VUG'

In [21]:
agent_executor.run(question_2)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "You are an agent designed to interact with a PostgreSQL database.\nGiven an input question, create a syntactically correct PostgresSQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (\") to denote them as delimited identifiers.\nPay attention to use only the column n

Error in MyCustomHandler.on_llm_end callback: TypeError("on_llm_end() got an unexpected keyword argument 'parent_run_id'")


[llm/end] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Thought: We need to get the corresponding information for the top 5 equity funds in terms of year to date returns.\nAction: sql_get_similar_examples\nAction Input: What are the corresponding inception date,risk level, expense ratio and average annual return since inception for these funds.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {},
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 2:chain:LLMChain] s] Exiting Chain run with output:
{
  "text": "Thought: We need to get the corresponding information for the top 5 equity funds in terms of year to date returns.\nAction: sql_get_similar_examples\nAction Input: What are the corresponding incepti

Error in MyCustomHandler.on_llm_end callback: TypeError("on_llm_end() got an unexpected keyword argument 'parent_run_id'")


[llm/end] [1:chain:AgentExecutor > 6:chain:LLMChain > 7:llm:OpenAI] [1.64s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " We can use the sql_query from the metadata to get the corresponding information for the top 5 equity funds in terms of year to date returns.\nAction: sql_db_query\nAction Input: SELECT fund_detail.inception_date, fund_detail.risk_level, fund_detail.expense_ratio, performance.fund_return_since_inception FROM fund_detail INNER JOIN performance ON performance.symbol = fund_detail.symbol WHERE fund_detail.symbol IN ('MGK', 'VMGAX', 'VGT', 'VITAX', 'VUG');",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {},
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 6:chain:LLMChain] [1.64s] Exiting Chain run with output:
{
  "text": " We can use the sql_qu

...

Error in MyCustomHandler.on_llm_end callback: TypeError("on_llm_end() got an unexpected keyword argument 'parent_run_id'")


[llm/end] [1:chain:AgentExecutor > 15:chain:LLMChain > 16:llm:OpenAI] [2.50s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " We need to format the results in a concise and easy to understand manner.\nAction: Format the results\nAction Input: [(datetime.date(2007, 12, 17), 'Moderate to Aggressive', 0.0007, 0.1204), (datetime.date(2008, 4, 3), 'Moderate to Aggressive', 0.0006, 0.1266), (datetime.date(2004, 3, 25), 'Aggressive', 0.001, 0.1348), (datetime.date(2004, 1, 26), 'Moderate to Aggressive', 0.0004, 0.1057), (datetime.date(2004, 1, 26), 'Aggressive', 0.001, 0.1276)]",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {},
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 15:chain:LLMChain] [2.50s] Exiting Chain run with output:
{
  "text": " We need to format the 

Error in MyCustomHandler.on_llm_end callback: TypeError("on_llm_end() got an unexpected keyword argument 'parent_run_id'")


[llm/end] [1:chain:AgentExecutor > 18:chain:LLMChain > 19:llm:OpenAI] [2.43s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " I now know the final answer\nFinal Answer: The corresponding inception date, risk level, expense ratio and average annual return since inception for the top 5 equity funds in terms of year to date returns are December 17, 2007, Moderate to Aggressive, 0.0007, and 0.1204; April 3, 2008, Moderate to Aggressive, 0.0006, and 0.1266; March 25, 2004, Aggressive, 0.001, and 0.1348; January 26, 2004, Moderate to Aggressive, 0.0004, and 0.1057; and January 26, 2004, Aggressive, 0.001, and 0.1276.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {},
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 18:chain:LLMChain] [2.43s] Exiting Chain run with out

'The corresponding inception date, risk level, expense ratio and average annual return since inception for the top 5 equity funds in terms of year to date returns are December 17, 2007, Moderate to Aggressive, 0.0007, and 0.1204; April 3, 2008, Moderate to Aggressive, 0.0006, and 0.1266; March 25, 2004, Aggressive, 0.001, and 0.1348; January 26, 2004, Moderate to Aggressive, 0.0004, and 0.1057; and January 26, 2004, Aggressive, 0.001, and 0.1276.'

In [23]:
agent_executor.run(question_3)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "You are an agent designed to interact with a PostgreSQL database.\nGiven an input question, create a syntactically correct PostgresSQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (\") to denote them as delimited identifiers.\nPay attention to use only the column n

Error in MyCustomHandler.on_llm_end callback: TypeError("on_llm_end() got an unexpected keyword argument 'parent_run_id'")


[llm/end] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Thought: We need to query for funds that are classified as fixed income and order them by their year to date returns.\nAction: sql_get_similar_examples\nAction Input: What are the top 5 fixed income funds in terms of year to date returns.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {},
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 2:chain:LLMChain] s] Exiting Chain run with output:
{
  "text": "Thought: We need to query for funds that are classified as fixed income and order them by their year to date returns.\nAction: sql_get_similar_examples\nAction Input: What are the top 5 fixed income funds in terms of year to date returns."
}
[tool/st

Error in MyCustomHandler.on_llm_end callback: TypeError("on_llm_end() got an unexpected keyword argument 'parent_run_id'")


[llm/end] [1:chain:AgentExecutor > 6:chain:LLMChain > 7:llm:OpenAI] [1.51s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " We can use the sql_query metadata from the first document to construct our final query.\nAction: sql_db_query\nAction Input: SELECT fund_detail.symbol, performance.fund_return_ytd FROM fund_detail INNER JOIN performance ON performance.symbol = fund_detail.symbol WHERE (fund_detail.asset_class LIKE '%Money Market%' OR fund_detail.asset_class LIKE '%Bond%' OR fund_detail.asset_class LIKE '%Securities%') AND performance.fund_return_ytd IS NOT NULL ORDER BY performance.fund_return_ytd DESC LIMIT 5;",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {},
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 6:chain:LLMChain] [1.51s] Exiting Chain run with

Error in MyCustomHandler.on_llm_end callback: TypeError("on_llm_end() got an unexpected keyword argument 'parent_run_id'")


[llm/end] [1:chain:AgentExecutor > 9:chain:LLMChain > 10:llm:OpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " I now know the final answer\nFinal Answer: The top 5 fixed income funds in terms of year to date returns are VEGBX, VEMBX, VWEAX, VWEHX, and VMSAX.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {},
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 9:chain:LLMChain] s] Exiting Chain run with output:
{
  "text": " I now know the final answer\nFinal Answer: The top 5 fixed income funds in terms of year to date returns are VEGBX, VEMBX, VWEAX, VWEHX, and VMSAX."
}
[chain/end] [1:chain:AgentExecutor] [3.47s] Exiting Chain run with output:
{
  "output": "The top 5 fixed income funds in terms of year to date returns are VEGBX, VEMBX, VWEAX, VWEHX, and

'The top 5 fixed income funds in terms of year to date returns are VEGBX, VEMBX, VWEAX, VWEHX, and VMSAX.'

In [26]:
agent_executor.run(question_4)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "You are an agent designed to interact with a PostgreSQL database.\nGiven an input question, create a syntactically correct PostgresSQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (\") to denote them as delimited identifiers.\nPay attention to use only the column n

Error in MyCustomHandler.on_llm_end callback: TypeError("on_llm_end() got an unexpected keyword argument 'parent_run_id'")


[llm/end] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Thought: We need to query for the corresponding columns from the table \"fund_info\" for the funds mentioned in the previous answer.\nAction: entity_search\nAction Input: fixed income funds",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {},
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 2:chain:LLMChain] s] Exiting Chain run with output:
{
  "text": "Thought: We need to query for the corresponding columns from the table \"fund_info\" for the funds mentioned in the previous answer.\nAction: entity_search\nAction Input: fixed income funds"
}
[tool/start] [1:chain:AgentExecutor > 4:tool:entity_search] Entering Tool run with input:
"fixed income fu

Error in MyCustomHandler.on_llm_end callback: TypeError("on_llm_end() got an unexpected keyword argument 'parent_run_id'")


[llm/end] [1:chain:AgentExecutor > 6:chain:LLMChain > 7:llm:OpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " We can use the similar examples to adapt the query for the previous answer.\nAction: sql_get_similar_examples\nAction Input: What are the corresponding inception date,risk level, expense ratio and average annual return since inception for these funds",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {},
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 6:chain:LLMChain] s] Exiting Chain run with output:
{
  "text": " We can use the similar examples to adapt the query for the previous answer.\nAction: sql_get_similar_examples\nAction Input: What are the corresponding inception date,risk level, expense ratio and average annual return since inception f

...

Error in MyCustomHandler.on_llm_end callback: TypeError("on_llm_end() got an unexpected keyword argument 'parent_run_id'")


[llm/end] [1:chain:AgentExecutor > 28:chain:LLMChain > 29:llm:OpenAI] [1.61s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " We can use the column \"average_annual_fund_return_since_inception\" from the table \"performance\" to answer the question.\nAction: sql_db_query\nAction Input: \"SELECT fund_detail.inception_date, fund_detail.risk_level, fund_detail.expense_ratio, performance.average_annual_fund_return_since_inception FROM fund_detail INNER JOIN performance ON performance.symbol = fund_detail.symbol WHERE fund_detail.symbol IN ('VEGBX', 'VEMBX', 'VWEAX', 'VWEHX', 'VMSAX');\"",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {},
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 28:chain:LLMChain] [1.61s] Exiting Chain run with output:
{
  "text": " We can use

Error in MyCustomHandler.on_llm_end callback: TypeError("on_llm_end() got an unexpected keyword argument 'parent_run_id'")


[llm/end] [1:chain:AgentExecutor > 31:chain:LLMChain > 32:llm:OpenAI] [2.34s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " I now know the final answer\nFinal Answer: The corresponding inception dates for the top 5 fixed income funds are 2016-03-10, 2021-10-12, 1978-12-27, 2017-12-06, and 2001-11-12. The corresponding risk levels for the top 5 fixed income funds are Moderate, Moderate, Moderate, Moderate, and Moderate. The corresponding expense ratios for the top 5 fixed income funds are 0.0055, 0.003, 0.0023, 0.004, and 0.0013. The corresponding average annual returns since inception for the top 5 fixed income funds are 0.0582, -0.0186, 0.0774, 0.04, and 0.0587.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {},
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor 

'The corresponding inception dates for the top 5 fixed income funds are 2016-03-10, 2021-10-12, 1978-12-27, 2017-12-06, and 2001-11-12. The corresponding risk levels for the top 5 fixed income funds are Moderate, Moderate, Moderate, Moderate, and Moderate. The corresponding expense ratios for the top 5 fixed income funds are 0.0055, 0.003, 0.0023, 0.004, and 0.0013. The corresponding average annual returns since inception for the top 5 fixed income funds are 0.0582, -0.0186, 0.0774, 0.04, and 0.0587.'

In [27]:
agent_executor.run(question_5)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "You are an agent designed to interact with a PostgreSQL database.\nGiven an input question, create a syntactically correct PostgresSQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (\") to denote them as delimited identifiers.\nPay attention to use only the column n

Error in MyCustomHandler.on_llm_end callback: TypeError("on_llm_end() got an unexpected keyword argument 'parent_run_id'")


[llm/end] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Thought: We need to find the long-term bond funds and sort them by average annual return since inception.\nAction: sql_get_similar_examples\nAction Input: Find all long-term bond funds and return the top 5 in terms of average annual return since inception.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {},
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 2:chain:LLMChain] s] Exiting Chain run with output:
{
  "text": "Thought: We need to find the long-term bond funds and sort them by average annual return since inception.\nAction: sql_get_similar_examples\nAction Input: Find all long-term bond funds and return the top 5 in terms of average annual

Error in MyCustomHandler.on_llm_end callback: TypeError("on_llm_end() got an unexpected keyword argument 'parent_run_id'")


[llm/end] [1:chain:AgentExecutor > 6:chain:LLMChain > 7:llm:OpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " We can use the sql_get_similar_examples tool to find similar examples to adapt them to the user question.\nAction: entity_search\nAction Input: Long-term bond funds",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {},
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 6:chain:LLMChain] s] Exiting Chain run with output:
{
  "text": " We can use the sql_get_similar_examples tool to find similar examples to adapt them to the user question.\nAction: entity_search\nAction Input: Long-term bond funds"
}
[tool/start] [1:chain:AgentExecutor > 8:tool:entity_search] Entering Tool run with input:
"Long-term bond funds"
[tool/end] [1:chain:AgentExecutor > 8:too

...

Error in MyCustomHandler.on_llm_end callback: TypeError("on_llm_end() got an unexpected keyword argument 'parent_run_id'")


[llm/end] [1:chain:AgentExecutor > 16:chain:LLMChain > 17:llm:OpenAI] [1.60s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " We can now construct the final query.\nAction: sql_db_query\nAction Input: SELECT fund_detail.symbol, fund_detail.inception_date, fund_detail.risk_level, fund_detail.expense_ratio, performance.average_annual_fund_return_since_inception FROM fund_detail INNER JOIN performance ON performance.symbol = fund_detail.symbol WHERE fund_detail.asset_class LIKE '%Long-Term Bond%' AND performance.average_annual_fund_return_since_inception IS NOT NULL ORDER BY performance.average_annual_fund_return_since_inception DESC LIMIT 5;",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {},
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 16:chain:LLMChain] [1.60

Error in MyCustomHandler.on_llm_end callback: TypeError("on_llm_end() got an unexpected keyword argument 'parent_run_id'")


[llm/end] [1:chain:AgentExecutor > 19:chain:LLMChain > 20:llm:OpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " I now know the final answer.\nFinal Answer: The top 5 long-term bond funds in terms of average annual return since inception are VWESX, VWAHX, VUSTX, VWETX, and VWLTX.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {},
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 19:chain:LLMChain] s] Exiting Chain run with output:
{
  "text": " I now know the final answer.\nFinal Answer: The top 5 long-term bond funds in terms of average annual return since inception are VWESX, VWAHX, VUSTX, VWETX, and VWLTX."
}
[chain/end] [1:chain:AgentExecutor] [6.63s] Exiting Chain run with output:
{
  "output": "The top 5 long-term bond funds in terms of average annu

'The top 5 long-term bond funds in terms of average annual return since inception are VWESX, VWAHX, VUSTX, VWETX, and VWLTX.'

In [28]:
agent_executor.run(question_6)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "You are an agent designed to interact with a PostgreSQL database.\nGiven an input question, create a syntactically correct PostgresSQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (\") to denote them as delimited identifiers.\nPay attention to use only the column n

Error in MyCustomHandler.on_llm_end callback: TypeError("on_llm_end() got an unexpected keyword argument 'parent_run_id'")


[llm/end] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] [1.00s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Thought: We need to query the database for funds that are classified as small blend and then sort them by their average annual return since inception.\nAction: sql_get_similar_examples\nAction Input: Find all small blend funds and return the top 5 in terms of average annual return since inception.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {},
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 2:chain:LLMChain] [1.00s] Exiting Chain run with output:
{
  "text": "Thought: We need to query the database for funds that are classified as small blend and then sort them by their average annual return since inception.\nAction: sql_get_similar_exam

Error in MyCustomHandler.on_llm_end callback: TypeError("on_llm_end() got an unexpected keyword argument 'parent_run_id'")


[llm/end] [1:chain:AgentExecutor > 6:chain:LLMChain > 7:llm:OpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " These examples are not exactly what we need, but they give us an idea of how to construct our query.\nAction: entity_search\nAction Input: small blend funds",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {},
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 6:chain:LLMChain] s] Exiting Chain run with output:
{
  "text": " These examples are not exactly what we need, but they give us an idea of how to construct our query.\nAction: entity_search\nAction Input: small blend funds"
}
[tool/start] [1:chain:AgentExecutor > 8:tool:entity_search] Entering Tool run with input:
"small blend funds"
[tool/end] [1:chain:AgentExecutor > 8:tool:entity_search] s]

...

Error in MyCustomHandler.on_llm_end callback: TypeError("on_llm_end() got an unexpected keyword argument 'parent_run_id'")


[llm/end] [1:chain:AgentExecutor > 16:chain:LLMChain > 17:llm:OpenAI] [1.60s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " The correct column name for average annual return since inception is \"average_annual_fund_return_since_inception\".\nAction: sql_db_query\nAction Input: SELECT fund_detail.symbol, performance.average_annual_fund_return_since_inception FROM fund_detail INNER JOIN performance ON performance.symbol = fund_detail.symbol WHERE fund_detail.category LIKE '%Small Blend%' AND performance.average_annual_fund_return_since_inception IS NOT NULL ORDER BY performance.average_annual_fund_return_since_inception DESC LIMIT 5;",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {},
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 16:chain:LLMChain] [1.61s] Exi

Error in MyCustomHandler.on_llm_end callback: TypeError("on_llm_end() got an unexpected keyword argument 'parent_run_id'")


[llm/end] [1:chain:AgentExecutor > 19:chain:LLMChain > 20:llm:OpenAI] [1.18s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " These are the top 5 small blend funds in terms of average annual return since inception.\nThought: I now know the final answer\nFinal Answer: The top 5 small blend funds in terms of average annual return since inception are VIOO, VTMSX, VSCPX, VTSIX, and VSMSX.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {},
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 19:chain:LLMChain] [1.18s] Exiting Chain run with output:
{
  "text": " These are the top 5 small blend funds in terms of average annual return since inception.\nThought: I now know the final answer\nFinal Answer: The top 5 small blend funds in terms of average annual return since in

'The top 5 small blend funds in terms of average annual return since inception are VIOO, VTMSX, VSCPX, VTSIX, and VSMSX.'

In [29]:
agent_executor.run(question_7)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Find all the funds that has less than 5% allocated to Emerging Markets region and at least 10% exposure to the health care sector.",
  "history": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Find all the funds that has less than 5% allocated to Emerging Markets region and at least 10% exposure to the health care sector.",
  "history": [],
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "You are an agent designed to interact with a PostgreSQL database.\nGiven an input question, create a syntactically correct PostgresSQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples

'VGWLX, VGWAX, VGYAX, VGWIX'

In [30]:
agent_executor.run(question_8)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "You are an agent designed to interact with a PostgreSQL database.\nGiven an input question, create a syntactically correct PostgresSQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (\") to denote them as delimited identifiers.\nPay attention to use only the column n

'No such funds.'

In [31]:
agent_executor.run(question_10)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "You are an agent designed to interact with a PostgreSQL database.\nGiven an input question, create a syntactically correct PostgresSQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (\") to denote them as delimited identifiers.\nPay attention to use only the column n

'VGWAX, VGWLX'

In [10]:
agent_executor.run('Provide me with a regional breakdown and sectoral breakdown of these funds')

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "You are an agent designed to interact with a PostgreSQL database.\nGiven an input question, create a syntactically correct PostgresSQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (\") to denote them as delimited identifiers.\nPay attention to use only the column n

'The regional breakdown and sectoral breakdown of the funds VGWAX and VGWLX are Emerging Markets, Europe, Pacific, North America, Communication Services, Consumer Discretionary, Consumer Staples, Energy, Financials, Health Care, Industrials, Information Technology, Materials, Real Estate, and Utilities.'

In [11]:
agent_executor.run('Please provide me with the allocation for each region and sectors.')

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "You are an agent designed to interact with a PostgreSQL database.\nGiven an input question, create a syntactically correct PostgresSQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (\") to denote them as delimited identifiers.\nPay attention to use only the column n

'The funds VGWAX and VGWLX have the following allocation for each region and sector: Emerging Markets: 0.011, Europe: 0.248, Pacific: 0.134, North America: 0.607, Communication Services: 0.051, Consumer Discretionary: 0.068, Consumer Staples: 0.063, Energy: 0.059, Financials: 0.207, Health Care: 0.156, Industrials: 0.131, Information Technology: 0.147, Materials: 0.027, Real Estate: 0.026, and Utilities: 0.065.'

In [9]:
import pandas as pd

In [15]:
region_df = pd.read_csv('../data/processed/20231212/region.csv')

In [18]:
sector_df = pd.read_csv('../data/processed/20231212/sector.csv')